In [175]:
import ibis as ib

# Connect to a local DuckDB database file
conn = ib.duckdb.connect(database='star_wars.duckdb')

# Print all the tables in the database
print(conn.list_tables())

# Lets look at the characters table using Ibis
# To generate an Ibis Table:
table = conn.table("characters")
print(table)

['battles', 'characters', 'cities', 'droids', 'events', 'films', 'music', 'organizations', 'planets', 'quotes', 'species', 'starships', 'timeline', 'vehicles', 'weapons']
DatabaseTable: characters
  id          int64
  name        string
  species     string
  gender      string
  height      float64
  weight      float64
  hair_color  string
  eye_color   string
  skin_color  string
  year_born   float64
  homeworld   string
  year_died   float64
  description string


In [176]:
ib.to_sql(table)

```sql
SELECT
  *
FROM "characters"
```

In [177]:
# We can only see the schema until we use an action method on the table.
# This is because Ibis is lazy and only evaluates when needed, not when the object is created.

head_expr = table.head()
head_expr

r0 := DatabaseTable: characters
  id          int64
  name        string
  species     string
  gender      string
  height      float64
  weight      float64
  hair_color  string
  eye_color   string
  skin_color  string
  year_born   float64
  homeworld   string
  year_died   float64
  description string

Limit[r0, n=5]

In [178]:
# To evaulate it we would need to execute it with .execute(), which returns the table as a pandas dataframe
head = head_expr.execute()
head

,id,name,species,gender,height,weight,hair_color,eye_color,skin_color,year_born,homeworld,year_died,description
0,1,Luke Skywalker,Human,Male,1.72,77.0,Blond,Blue,Light,19.0,Tatooine,34.0,The main protagonist of the original trilogy.
1,2,Leia Organa,Human,Female,1.50,49.0,Brown,Brown,Light,19.0,Alderaan,35.0,A leader in the Rebel Alliance and twin sister...
2,3,Darth Vader,Human,Male,2.02,136.0,None,Yellow,Pale,41.0,Tatooine,4.0,The Sith Lord formerly known as Anakin Skywalker.
3,4,Yoda,Yoda's species,Male,0.66,17.0,White,Brown,Green,896.0,Unknown,4.0,A wise and powerful Jedi Master.
4,5,Han Solo,Human,Male,1.80,80.0,Brown,Hazel,Light,29.0,Corellia,34.0,A smuggler turned hero in the Rebel Alliance.


In [179]:
print(type(table))
print(type(head))
# To get something similar to df.shape, do this:
(table.count().execute(), len(table.schema()))

<class 'ibis.expr.types.relations.Table'>
<class 'pandas.core.frame.DataFrame'>


(np.int64(96), 13)

Ibis has several aggregate functions available to help summarize data without storing all of the data in memory (sum, min, max, mean, etc.). If we wanted to do this with Pandas, the whole data set would have to be stored in memory.

In [180]:
# create an average weight table expression
average_weight_expr = table["weight"].mean()
# You can also do table.weight.mean()

#print out the raw SQL of the ibis table expression
print(ib.to_sql(average_weight_expr))
# Execute the expression to get the result
avg_weight = average_weight_expr.execute()
print(f"Average weight of every star wars character is {avg_weight} kg.")


SELECT
  AVG("t0"."weight") AS "Mean(weight)"
FROM "characters" AS "t0"
Average weight of every star wars character is 87.94565217391305 kg.


### Group by and Aggregate

We can compute multiple aggregate functions at once and have a dataframe returned like we did with finches.csv on our intro_pandas.ipynb notes.

In [181]:
group_agg_expr = table.group_by(table["species"]).aggregate(mean_height = table["height"].mean(), mean_weight = table["weight"].mean(),  max_weight = table["weight"].max())
print(ib.to_sql(group_agg_expr))
group_agg_expr.execute()

SELECT
  "t0"."species",
  AVG("t0"."height") AS "mean_height",
  AVG("t0"."weight") AS "mean_weight",
  MAX("t0"."weight") AS "max_weight"
FROM "characters" AS "t0"
GROUP BY
  1


,species,mean_height,mean_weight,max_weight
0,Gungan,1.960000,66.00,66.0
1,Ardennian,1.400000,40.00,40.0
2,Unknown,1.710000,65.50,100.0
3,Kel Dor,1.880000,80.00,80.0
4,Besalisk,2.500000,300.00,300.0
5,Chiss,1.950000,85.00,85.0
6,Yoda's species,0.660000,17.00,17.0
7,Twi'lek,1.817500,57.50,65.0
8,Hutt,3.900000,1358.00,1358.0
9,Sullustan,1.600000,70.00,70.0


### Chaining expressions
We can keep chaining expressions until we are satisfied with the results we have. In this case, if I wanted to order the previous expression by mean_height in descending order, I don't have to start from table, I could start from the last expression.

In [182]:
sorted_agg_expr = group_agg_expr.order_by(ib.desc(group_agg_expr["mean_height"]))
sorted_agg_expr.execute()

,species,mean_height,mean_weight,max_weight
0,Hutt,3.900000,1358.00,1358.0
1,Besalisk,2.500000,300.00,300.0
2,Wookiee,2.280000,112.00,112.0
3,Nautolan,1.960000,87.00,87.0
4,Gungan,1.960000,66.00,66.0
5,Chiss,1.950000,85.00,85.0
6,Trandoshan,1.900000,113.00,113.0
7,Kel Dor,1.880000,80.00,80.0
8,Togruta,1.875000,61.50,68.0
9,Dathomirian,1.860000,70.00,100.0


We can select and filter only the data we want to analyze and have a dataframe returned to shorten the amount of memory used significantly. For example, if this data set was large and we only cared about certain columns, or rows of weights that are > 65 kgs, we can filter and select that data from the database and get a Pandas dataframe out of that. We can also get a random sample of data if we don't want to filter our data.

In [183]:
# With our original Ibis table object "table," we can select only the columns we need
select = table.select("name", "species", "gender", "height", "weight")
# Then we can filter only the weights > 50 from that select table expression
filtered = select.filter(select.weight > 65)
fdf = filtered.execute()
fdf


,name,species,gender,height,weight
0,Luke Skywalker,Human,Male,1.72,77.0
1,Darth Vader,Human,Male,2.02,136.0
2,Han Solo,Human,Male,1.80,80.0
3,Chewbacca,Wookiee,Male,2.28,112.0
4,Obi-Wan Kenobi,Human,Male,1.82,81.0
...,...,...,...,...,...
60,Galen Erso,Human,Male,1.83,75.0
61,Saw Gerrera,Human,Male,1.78,75.0
62,Admiral Raddus,Mon Calamari,Male,1.80,72.0
63,Dryden Vos,Human,Male,1.85,80.0


In [184]:
# Sampling can give you amount of rows if int or percentage of rows if float
sample = select.sample(.5) # getting a random 50% of data
sdf = sample.execute()
sdf

,name,species,gender,height,weight
0,Luke Skywalker,Human,Male,1.72,77.0
1,Leia Organa,Human,Female,1.50,49.0
2,Han Solo,Human,Male,1.80,80.0
3,Chewbacca,Wookiee,Male,2.28,112.0
4,Padmé Amidala,Human,Female,1.65,45.0
5,Mace Windu,Human,Male,1.88,84.0
6,Jar Jar Binks,Gungan,Male,1.96,66.0
7,Rey,Human,Female,1.70,54.0
8,Kylo Ren,Human,Male,1.89,89.0
9,C-3PO,Droid,None,1.71,NaN
